In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, isnull, trim, avg, count, sum
from pyspark.sql.types import StringType, IntegerType
df_bancos = spark.read.option("delimiter", "\t").option("header", True).option("charset", "latin1").csv('s3://denis-ingestao-bancos-grupo1/banco/bancos.tsv')
column_to_rename_old = ['Segmento','CNPJ','Nome']
column_to_rename_new = ['segmento','cnpj','nome']
for i in range(len(column_to_rename_old)):
    df_bancos = df_bancos.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
df_bancos = df_bancos.withColumn("nome", regexp_replace("nome", " - PRUDENCIAL", ""))
df_bancos.filter(df_bancos.cnpj == "0").show(truncate=False)
df_empregados_01 = spark.read.option("delimiter", "|").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/empregados/glassdoor_consolidado_join_match_less_v2.csv')
df_empregados_02 = spark.read.option("delimiter", "|").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/empregados/glassdoor_consolidado_join_match_v2.csv')
column_to_rename_old = ['employer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer-website', 'employer-headquarters', 'employer-founded', 'employer-industry', 'employer-revenue', 'url', 'Geral', 'Cultura e valores', 'Diversidade e inclusao', 'Qualidade de vida', 'Alta lideranca', 'Remuneracao e beneficios', 'Oportunidades de carreira', 'Recomendam para outras pessoas(%)', 'Perspectiva positiva da empresa(%)', 'CNPJ', 'Nome', 'match_percent']
column_to_rename_new = ['employer_name', 'reviews_count', 'culture_count', 'salaries_count', 'benefits_count', 'employer_website', 'employer_headquarters', 'employer_founded', 'employer_industry', 'employer_revenue', 'url', 'geral', 'cultura_e_valores', 'diversidade_e_inclusao', 'qualidade_de_vida', 'alta_lideranca', 'remuneracao_e_beneficios', 'oportunidades_de_carreira', 'recomendam_para_outras_pessoas_perc', 'perspectiva_positiva_da_empresa_perc', 'cnpj_emp', 'nome_emp', 'match_percent']

for i in range(len(column_to_rename_old)):
    df_empregados_01 = df_empregados_01.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_empregados_02 = df_empregados_02.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])

df_empregados_01.filter(df_empregados_01.nome_emp == "APE POUPEX").show(truncate=False)
df_empregados_02.show(truncate=False)
columns = ['nome_emp', col("geral").cast("float").alias("geral"),col("remuneracao_e_beneficios").cast("float").alias("remuneracao_e_beneficios")]

df_empregados = df_empregados_01.select(columns).union(df_empregados_02.select(columns))
df_empregados.show(truncate=False)
df_reclamacoes_202101 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2021_tri_01.csv')
df_reclamacoes_202102 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2021_tri_02.csv')
df_reclamacoes_202103 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2021_tri_03.csv')
df_reclamacoes_202104 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2021_tri_04.csv')
df_reclamacoes_202201 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2021_tri_01.csv')
df_reclamacoes_202202 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2022_tri_01.csv')
df_reclamacoes_202203 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2022_tri_03.csv')
df_reclamacoes_202204 = spark.read.option("delimiter", ";").option("header", True).csv('s3://denis-ingestao-bancos-grupo1/reclamacao/2022_tri_04.csv')
column_to_rename_old = ['Ano', 'Trimestre', 'Categoria', 'Tipo', 'CNPJ IF', 'Instituicao financeira', 'indice', 'Quantidade de reclamacoes reguladas procedentes', 'Quantidade de reclamacoes reguladas - outras', 'Quantidade de reclamacoes nno reguladas', 'Quantidade total de reclamacoes', 'Quantidade total de clientes - CCS e SCR', 'Quantidade de clientes - CCS', 'Quantidade de clientes - SCR',',,']
column_to_rename_new = ['ano', 'trimestre', 'categoria', 'tipo', 'cnpj_if', 'instituicao_financeira', 'indice', 'quantidade_de_reclamacoes_reguladas_procedentes', 'quantidade_de_reclamacoes_reguladas_outras', 'quantidade_de_reclamacoes_nao_reguladas', 'quantidade_total_de_reclamacoes', 'quantidade_total_de_clientes_ccs_e_scr', 'quantidade_de_clientes_ccs', 'quantidade_de_clientes_scr', ',,']
for i in range(len(column_to_rename_old)):
    df_reclamacoes_202101 = df_reclamacoes_202101.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202102 = df_reclamacoes_202102.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202103 = df_reclamacoes_202103.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202104 = df_reclamacoes_202104.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202201 = df_reclamacoes_202201.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202202 = df_reclamacoes_202202.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202203 = df_reclamacoes_202203.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
    df_reclamacoes_202204 = df_reclamacoes_202204.withColumnRenamed(column_to_rename_old[i], column_to_rename_new[i])
columns = ['ano', 'trimestre','instituicao_financeira','indice','quantidade_total_de_reclamacoes','quantidade_total_de_clientes_ccs_e_scr']

df_reclamacoes = df_reclamacoes_202101.select(columns).union(df_reclamacoes_202102.select(columns))\
.union(df_reclamacoes_202103.select(columns)).union(df_reclamacoes_202104.select(columns)).union(df_reclamacoes_202201.select(columns))\
.union(df_reclamacoes_202202.select(columns)).union(df_reclamacoes_202203.select(columns)).union(df_reclamacoes_202204.select(columns))
df_reclamacoes.show(truncate=False)
df_reclamacoes = df_reclamacoes.withColumn("instituicao_financeira", regexp_replace("instituicao_financeira", "conglomerado", ""))
df_reclamacoes = df_reclamacoes.withColumn("instituicao_financeira", regexp_replace("instituicao_financeira","[()]",""))
df_reclamacoes = df_reclamacoes.withColumn("instituicao_financeira",trim(col("instituicao_financeira")))
df_reclamacoes = df_reclamacoes.withColumn("indice", regexp_replace("indice",",","."))
df_reclamacoes.show(truncate=False)
df_reclamacoes.filter(df_reclamacoes.instituicao_financeira == "ABC-BRASIL").show(truncate=False)
columns = ['segmento','cnpj','nome','ano','trimestre',col("indice").cast('float').alias("indice"),col("quantidade_total_de_reclamacoes").cast('int').alias("quantidade_total_de_reclamacoes"),col("quantidade_total_de_clientes_ccs_e_scr").cast('int').alias("quantidade_total_de_clientes_ccs_e_scr")]

df_bancos_reclamacoes = df_bancos.join(df_reclamacoes,df_bancos.nome ==  df_reclamacoes.instituicao_financeira,"inner").select(columns)
df_bancos_reclamacoes.show(truncate=False)
columns = ['segmento','cnpj','nome',col("geral").cast('float').alias("geral"),col("remuneracao_e_beneficios").cast('float').alias("remuneracao_e_beneficios")]

df_bancos_empregados = df_bancos.join(df_empregados,df_bancos.nome ==  df_empregados.nome_emp,"inner").select(columns).distinct()
df_bancos_empregados.show(truncate=False)
df_bancos_reclamacoes_agg = df_bancos_reclamacoes.groupBy("nome").agg(avg("indice").alias("indice"), sum("quantidade_total_de_reclamacoes").alias("quantidade_total_de_reclamacoes") \
                      , avg("quantidade_total_de_clientes_ccs_e_scr").alias("quantidade_total_de_clientes_ccs_e_scr"))
df_bancos_reclamacoes_agg = df_bancos_reclamacoes_agg.withColumnRenamed("nome", "nome_rec")
df_bancos_reclamacoes_agg.show(truncate=False)
columns = [col("nome").alias("nome_banco"),"cnpj",col("segmento").alias("classificacao_banco"),\
           col("quantidade_total_de_clientes_ccs_e_scr").alias("quantidade_de_clientes_do_bancos"),col("indice").alias("indice_de_reclamacoes"),\
           col("quantidade_total_de_reclamacoes").alias("quantidade_de_reclamacoes"),col("geral").alias("indice_de_satisfacao_dos_funcionarios_dos_bancos"),\
           col("remuneracao_e_beneficios").alias("indice_de_satisfacao_com_salarios_dos_funcionários_dos_bancos")]

df_fato = df_bancos_empregados.join(df_bancos_reclamacoes_agg,df_bancos_empregados.nome ==  df_bancos_reclamacoes_agg.nome_rec,"inner").select(columns).distinct()
df_fato.show()
df_fato.write.format("hive").mode("append").saveAsTable("bancos_classif.fato")
spark.sql("SELECT * FROM bancos_classif.fato").show()
job.commit()